In [1]:
class Patient:
    def __init__(self,iid, has_covid19: bool, arrival_time: int, boredom: int):
        self.id = iid
        self.has_covid19 = has_covid19
        self.arrival_time = arrival_time #Omadan to Bimarestan
        self.delivered_time = -1 #Raftan to Saf Doctor
        self.triage_time = -1
        self.inspection_time = -1
        self.finish_time = -1
        self.boredom = boredom
        self.bored = False
    
    def is_bored(self, clock) -> bool:
        self.bored = (clock - self.arrival_time > self.boredom)
        return self.bored
    
    def __repr__(self):
        return "ID: " + str(self.id) + \
        ", Arrival Time: " + str(self.arrival_time) + ", Boredom: " + str(self.boredom) + \
        ", Has COVID19: " + str(self.has_covid19) + ", Delivered Time: " + str(self.delivered_time) + \
        ", Triage Time: " + str(self.triage_time) + ", Inspection Time: " + str(self.inspection_time) + \
        ", Finish Time: " + str(self.finish_time)
    
    def csv(self):
        obj = []
        obj.append(str(self.id))
        obj.append(str(self.has_covid19))
        obj.append(str(self.arrival_time))
        obj.append(str(self.triage_time))
        obj.append(str(self.delivered_time))
        obj.append(str(self.inspection_time))
        obj.append(str(self.finish_time))
        obj.append(str(self.boredom))
        obj.append(str(self.bored))
        return ",".join(obj)

In [5]:
import queue

class HQueue:
    counter = 1
    def __init__(self):
        self.iid = HQueue.counter
        HQueue.counter += 1
        self.covid19_queue = queue.Queue()
        self.others_queue = queue.Queue()
    
    def get(self, clock) -> Patient:
        while not self.covid19_queue.empty():
            patient = self.covid19_queue.get()
            if not patient.is_bored(clock):
                return patient
            else:
                expose_metric(patient.csv(), "patients.csv")
        while not self.others_queue.empty():
            patient = self.others_queue.get()
            if not patient.is_bored(clock):
                return patient
            else:
                expose_metric(patient.csv(), "patients.csv")
        return None
    
    def put(self, patient: Type[Patient]):
        if patient.has_covid19:
            self.covid19_queue.put(patient)
        else:
            self.others_queue.put(patient)
    
    def size(self) -> int:
        return self.covid19_queue.qsize() + self.others_queue.qsize()

In [6]:
class MockInput:
    iid = 1
    def __init__(self, alpha, lam):
        self.bordemAverage = alpha
        self.lam = lam
        
    def create_patient(self, lastArriaval):
        arriaval = lastArriaval
        has_covid19 = False
        if random.uniform() > 0.9:
            has_covid19 = True
        bordemTime = round(random.exponential(scale=self.bordemAverage))
        patient = Patient(MockInput.iid, has_covid19=has_covid19, arrival_time=arriaval, boredom=bordemTime)
        MockInput.iid += 1
        arriaval += random.poisson(lam=self.lam)
        return patient, arriaval

In [7]:
from typing import List, Type
from numpy import random

class Triage:
    ii = 0
    def __init__(self, miu: float, inQ: Type[HQueue], outQueues: List[queue.Queue]):
        self.miu = miu
        self.inQ = inQ
        self.outQueues = outQueues
        self.idleTime = 0
        self.busy = -1
        self.currentPatient = None
          
    def preferedQueue(self) -> Type[queue.Queue]:
        minQSize = 99999999999999999999999
        Q = []
        for q in self.outQueues:
            if q.size() < minQSize:
                minQSize = q.size()
                Q = q
        return Q
    
    def process(self, clock):
        if self.busy > 0:
            self.busy -= 1
            if self.busy == 0:
                Q = self.preferedQueue()
                Q.put(self.currentPatient)
            return
        if self.inQ.size() == 0:
            self.idleTime += 1
            return
        patient = self.inQ.get(clock)
        if patient is None:
            self.idleTime += 1
            return
        Triage.ii += 1
        process = random.poisson(lam=self.miu) + 1
        patient.triage_time = clock
        patient.delivered_time = clock + process
        self.currentPatient = patient
        self.busy = process
        

In [8]:
class Doctor:
    def __init__(self, mean: float):
        self.mean = mean
        self.busy = -1
        self.idleTime = 0
        self.currentPatient = None
        
    def process(self, inQ, clock):
        if self.busy > 0:
            self.busy -= 1
            if self.busy == 0:
                expose_metric(self.currentPatient.csv(), "patients.csv")
            return
        if inQ.size() == 0:
            self.idleTime += 1
            return
        patient = inQ.get(clock)
        if patient is None:
            self.idleTime += 1
            return
        patient.inspection_time = clock
        process = round(random.exponential(scale=self.mean)) + 1
        patient.finish_time = process + clock
        self.busy = process
        self.currentPatient = patient
        

In [9]:
class Room:
    def __init__(self, service_rates: List[float], inQ: Type[HQueue]):
        self.doctors = []
        self.inQ = inQ
        for rate in service_rates:
            self.doctors.append(Doctor(rate))
    
    def getDoctors(self) -> List[Doctor]:
        return self.doctors
    
    def process(self, clock):
        for doc in self.doctors:
            doc.process(self.inQ, clock)
            

In [10]:
cash = {}
cash["patients.csv"] = []
cash["room.csv"] = []
cash["triage.csv"] = []
cash["patients.csv"].append("id,covid19,arriaval,triage,roomQ,inspection,finish,borredom,bored")
cash["room.csv"].append("id,clock,covid19,others,total")
cash["triage.csv"].append("id,clock,covid19,others,total")
def expose_metric(metric, filename):
    if filename not in cash:
        cash[filename] = []
    cash[filename].append(metric)
    if len(cash[filename]) > 1000:
        flush(filename)

def flush(filename):
    with open(filename, 'a') as f:
        for item in cash[filename]:
            f.write("%s\n" % item)
            cash[filename] = []

In [21]:
def exposeQueueMetric(triageQueue, outQueues, clock):
    triageQMetric = str(triageQueue.iid) + "," + str(clock) + "," + str(triageQueue.covid19_queue.qsize()) + "," + str(triageQueue.others_queue.qsize()) + "," + str(triageQueue.size())
    expose_metric(triageQMetric, "triage.csv")
    for q in outQueues:
        strq = str(q.iid) + "," + str(clock) + "," + str(q.covid19_queue.qsize()) + "," + str(q.others_queue.qsize()) + "," +str(q.size())
        expose_metric(strq, "room.csv")


def emptyQs(triageQueue, outQueues):
    a = True
    for q in outQueues:
        a = a and q.size() == 0
    return a and triageQueue.size() == 0
        
def hospital(alpha, lam, miu, outQueues: List[HQueue], maximum, rooms):
    clock = 0
    arriaval = 0 
    mockInput = MockInput(alpha=alpha, lam=lam)
    triageQueue = HQueue()
    triage = Triage(miu, triageQueue, outQueues)
    count = 0
    closed = False
    while count < maximum or (not emptyQs(triageQueue, outQueues)):
        if arriaval <= clock and count < maximum:
            patient, arriaval = mockInput.create_patient(arriaval)
            triageQueue.put(patient)
            count += 1
        triage.process(clock)
        for room in rooms:
            room.process(clock)
        exposeQueueMetric(triageQueue, outQueues, clock)
        clock += 1
qs = []
qs.append(HQueue())
qs.append(HQueue())
qs.append(HQueue())
r1 = Room([2, 3], qs[0])
r2 = Room([2, 3], qs[1])
r3 = Room([2, 3], qs[2])
hospital(alpha=40, lam=2.4, miu=1.1, outQueues=qs, maximum=10000, rooms=[r1, r2, r3])
flush("patients.csv")
flush("room.csv")
flush("triage.csv")